## This of the work shows how to preprocess data and perform smoothing operations on the H5AD format.

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(sys.path[0]))
import scanpy as sc
from EAGS_function import *
import anndata
import numpy as np
import pandas as pd

In [ ]:
def Normalize_using(adata):
    """
    Pre-processing of H5AD data
    """
    adata.var['mt']=adata.var_names.str.startswith(('MT-','mt-'))
    sc.pp.calculate_qc_metrics(adata,qc_vars=['mt'],inplace=True)
    adata=adata[adata.obs['pct_counts_mt']<10].copy()
    sc.pp.filter_cells(adata,min_counts=300)
    sc.pp.filter_genes(adata,min_cells=10)
    # max_counts=np.percentile(adata.X.sum(1),98.0)
    # sc.pp.filter_cells(adata,max_counts=max_counts)
    # raw_adata=adata.copy().raw.to_adata()
    raw_adata=adata.copy()
    sc.pp.normalize_per_cell(adata,counts_per_cell_after=1e4)
    sc.pp.log1p(adata)
    if type(adata.X)==sp.csr.csr_matrix or type(adata.X)==sp.csc.csc_matrix:
        adata.X = adata.X.toarray()
    adata.X = (adata.X - adata.X.mean(0)) / adata.X.std(0)
    return adata

In [ ]:
# Reading h5ad data and preprocessing it
adata_raw = sc.read_h5ad('raw.h5ad')
adata_norm = Normalize_using(adata_raw)
adata_norm.write_h5ad('norm.h5ad')

In [ ]:
# Processing of normalized h5ad data using EAGS
adata_GS = gaussian_smooth_adaptively(adata_norm, normalize_zscore=False)
adata_GS.write_h5ad('GS.h5ad')

## This part of the work is used to illustrate how to do various types of downstream analyses

In [ ]:
# After completing the cell annotations for Spatia ID or TANGRAM, place each single-cell label annotation into
# h5ad.obs['celltype_pred'] of the h5ad file
#
# Spatial-ID DOI：
# Shen R, Liu L, Wu Z, et al. Spatial-ID: a cell typing method for spatially resolved transcriptomics via transfer learning and spatial embedding. Nat Commun 2022;13:7640. doi:10.1038/s41467-022-35288-0.
#
# Tangram DOI：
# Biancalani T, Scalia G, Buffoni L, et al. Deep learning and alignment of spatially resolved single-cell transcriptomes with Tangram. Nat Methods 2021;18:1352–62. doi:10.1038/s41592-021-01264-7.

In [ ]:
# Spatial cell type map for cell annotation with Spatial-ID or Tangram
sc.pl.spatial(adata_GS, color='celltype_pred',spot_size=80)

In [ ]:
# calculating Calinski-Harabasz Index and Davies-Bouldin Index

import sklearn.metrics as skm
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score
sc.pp.pca(adata_GS,n_comps=50)
calinski_harabasz_score(adata_GS.obsm['X_pca'], adata_GS.obs['celltype_pred'])
davies_bouldin_score(adata_GS.obsm['X_pca'], adata_GS.obs['celltype_pred'])

# calculating Moran's I and Geary's C index of H5AD format file
import pandas as pd
sc.pp.neighbors(adata_GS, use_rep='spatial')

# here is one-hot coding of celltype, if you have deconvolution result, you can have a try
adata_GS.obsm['celltype_pred'] = pd.get_dummies(adata_GS.obs['celltype_pred'])
correction_mol = sc.metrics.morans_i(adata_GS, obsm='celltype_pred')
correction_gec = sc.metrics.gearys_c(adata_GS, obsm='celltype_pred')
correction_celltype = adata_GS.obsm['celltype_pred'].columns.to_list()

for j in range(len(correction_celltype)):
    if str(correction_celltype[j]) in adata_GS.obs['celltype_pred'].values.tolist():
        print('Cell Type: ', correction_celltype[j])
        print('Geary\'s C: ', correction_gec[j])
        print('Moran\'s l: ', correction_mol[j])

In [ ]:
# Making spatial maps of a single cell type
# Single-cell spatial type data corresponds to the corresponding published spatial cell atlas:
# https://scalablebrainatlas.incf.org/mouse/ABA_v3

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
cell_list = adata_GS.obs['celltype_pred'].value_counts().index.to_list()
for i in cell_list:
    sc.pl.spatial(adata_GS,color=["celltype_pred"],groups=i,spot_size=100,color_map='magma',title='EAGS')

In [ ]:
# Make the picture of Heatmap of non-zero ratio between the number of cell types and their marker genes obtained from raw and EAGS smoothed dataset
# Each cell type and its corresponding marker gene are saved in Adolescent.txt
celltype_name_list = ['TEGLU24', 'TEGLU7', 'MEINH2', 'MOL1','EPEN','VSMCA','DGGRC2','ACTE2']
import pandas as pd
data = pd.read_table('Adolescent.txt',sep=',', header=None)

markergene_dic = {}
for index, row in data.iterrows():
    list1 = row[1:].to_list()
    for i in range(len(list1)):
        list1[i] = list1[i][2:-1]
    markergene_dic[str(row[0])[1:-1]] = list1
marker_genes = []
for i in celltype_name_list:
    list = markergene_dic[i]
    print('cell:' + i + ' genes : ' + str(list))
    for j in list:
        marker_genes.append(j)

pd_concat = pd.DataFrame()
pd_concat_1 = pd.DataFrame()
pd_concat_2 = pd.DataFrame()
pd3 = pd.DataFrame()
for i in celltype_name_list:
    df1 = adata_norm[adata_norm.obs['celltype_pred'] == i,adata_norm.var_names.isin(marker_genes)].to_df()
    df2 = adata_GS[adata_GS.obs['celltype_pred'] == i,adata_GS.var_names.isin(marker_genes)].to_df()

    df1 = df1[marker_genes]
    df2 = df2[marker_genes]

    ## Count how many cells of a specific cell type are labeled before and after the interpolation.
    cellnum1 = df1.index.__len__()
    cellnum2 = df2.index.__len__()
    ## The percentage of cells expressing a particular markergene in a particular cell type before and after the interpolation was counted separately.
    # nonzero_sum_1 = df1[df1 !=0].sum()
    nonzero_num_1 = ((df1!=0)==True).sum(0)
    # nonzero_1 = pd.DataFrame(nonzero_sum_1 /nonzero_num_1,columns=['raw_' + i])

    # nonzero_sum_2 = df2[df2 !=0].sum()
    nonzero_num_2 = ((df2!=0)==True).sum(0)
    # nonzero_2 = pd.DataFrame(nonzero_sum_2 /nonzero_num_2,columns=['EAGS_' + i])

    pd1 = pd.DataFrame(nonzero_num_1/cellnum1,columns=['raw_' + i])
    pd2 = pd.DataFrame(nonzero_num_2/cellnum2,columns=['EAGS_' + i])
    pd_concat_1 = pd.concat([pd_concat_1,pd1],axis=1)
    pd_concat_2 = pd.concat([pd_concat_2,pd2],axis=1)
pd_concat = pd.concat([pd_concat_1,pd_concat_2],axis=1)
pd_concat = pd_concat.T

plt.style.use('seaborn-paper')
sns.set(font_scale=0.5)
pdf = sns.heatmap(pd_concat)
pdf = sns.clustermap(pd_concat)
pdf.figure.savefig('heatmap_norm_EAGS.pdf')

In [ ]:
# Make the heatmap of the spatial distribution of the number of MID counts for a given genotype
from heatmap_make import *
gene_list = ['Slc6a3','Slc18a3','Cartpt']
show_markerlist(adata_GS,
                gene_list,
                size=3,
                save_path='.',
                semititle='EAGS_')

In [ ]:
# Difference between the processed EAGS data and the original data calculated using L2-error
# For use on simulation datasets containing Ground Truth Counts generated by ScDesign3.
# ScDesign3 DOI:
# Song D, Wang Q, Yan G, et al. scDesign3 generates realistic in silico data for multimodal single-cell and spatial omics. Nat Biotechnol 2023; Nature Research; 2023;1-6. doi: https://doi.org/10.1038/s41587-023-01772-1.

import warnings
warnings.filterwarnings("ignore")
import scanpy as sc
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import scipy.sparse as sp
from numpy import *

true_counts = adata_norm.X
GS_counts = adata_GS.X

TrueData = scaler.fit_transform(true_counts)
imputedData = scaler.fit_transform(GS_counts)
Datadiff=imputedData-TrueData
L2_error=np.linalg.norm(Datadiff)
print('L2-error between True and GS counts is ' + str(L2_error))